In [16]:
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd
import astropy.units as u
from astropy.coordinates import SkyCoord
from astropy.units import Quantity
from astroquery.gaia import Gaia
from astropy.table import Table

In [21]:
# query = """
# SELECT g.source_id, g.ra, g.dec, parallax, parallax_error,
#        pmra, pmra_error, pmdec, pmdec_error,
#        phot_g_mean_mag, phot_g_mean_flux_over_error,
#        phot_bp_mean_mag, phot_bp_mean_flux_over_error,
#        phot_rp_mean_mag, phot_rp_mean_flux_over_error,
#        phot_g_mean_mag + 5.0*log10(parallax/100.0) AS M_G, bp_rp,
#        s.ra,s.dec, u_mag, u_mag_error, g_mag, g_mag_error, r_mag, r_mag_error, i_mag, i_mag_error, z_mag, z_mag_error, clean_flag,
#        pan.ra,pan.dec,g_mean_psf_mag,g_mean_psf_mag_error,r_mean_psf_mag,r_mean_psf_mag_error,i_mean_psf_mag,i_mean_psf_mag_error,z_mean_psf_mag,z_mean_psf_mag_error,y_mean_psf_mag,y_mean_psf_mag_error,s.sdssdr9_oid

# FROM   gaiadr3.gaia_source AS g, gaiadr1.sdssdr9_original_valid AS s, gaiadr2.sdssdr9_best_neighbour AS xs,
#        gaiadr2.panstarrs1_original_valid AS pan, gaiadr3.panstarrs1_best_neighbour AS xpan
# WHERE g.source_id = xs.source_id AND s.sdssdr9_oid = xs.sdssdr9_oid
#   AND g.source_id = xpan.source_id AND pan.obj_id = xpan.original_ext_source_id
#   AND parallax > 10.0 AND parallax_over_error > 10.0
#   AND phot_bp_mean_flux_over_error > 10.0
#   AND phot_rp_mean_flux_over_error > 10.0
#   AND phot_g_mean_mag + 5.0*log10(parallax/100.0) > (3.333333*(phot_bp_mean_mag - phot_rp_mean_mag) + 8.333333)
#   AND (astrometric_sigma5d_max < 1.5 OR (ruwe <= 1.1 AND ipd_gof_harmonic_amplitude < 1))
#   AND ((phot_bp_n_obs >2 AND phot_rp_n_obs >2) OR phot_g_mean_mag < 19)
#   AND ((astrometric_excess_noise_sig < 2 OR (astrometric_excess_noise_sig >= 2 AND astrometric_excess_noise < 1.5)) OR astrometric_params_solved < 32)
# """

In [22]:
# query_data = Gaia.launch_job_async(query)
# query_result = query_data.get_results()
# gaia_data = query_result.to_pandas()
# print(gaia_data)
# sdssids = gaia_data['sdssdr9_oid']
# objidstr = ",".join(str(obj_id) for obj_id in sdssids)
# print(objidstr)

In [23]:
# gaia_data.to_csv('gaia_query.csv')

In [29]:
gaia_data = pd.read_csv('gaia_query.csv')

In [36]:
montreal_df = pd.read_csv('/home/jovyan/Final_Project/MWDD-export.csv')
montreal_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 17057 entries, 0 to 17056
Data columns (total 7 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   wdid      17057 non-null  object 
 1   icrsra    17057 non-null  object 
 2   icrsdec   17057 non-null  object 
 3   spectype  15807 non-null  object 
 4   teff      15051 non-null  float64
 5   logg      15024 non-null  float64
 6   Dpc       17057 non-null  float64
dtypes: float64(3), object(4)
memory usage: 932.9+ KB


In [33]:
gaia_ra_numpy = gaia_data['ra'].to_numpy()
gaia_dec_numpy = gaia_data['dec'].to_numpy()

In [37]:
montreal_ra_numpy = montreal_df['icrsra'].to_numpy()
montreal_dec_numpy = montreal_df['icrsdec'].to_numpy()

In [56]:
coords = SkyCoord(ra=montreal_ra_numpy,dec=montreal_dec_numpy, unit=(u.hourangle,u.deg))

In [50]:
coords = SkyCoord(ra=montreal_df['icrsra'].to_numpy(),
dec=montreal_df['icrsdec'].to_numpy(), unit=(u.hourangle,u.deg))

In [58]:
gaia_coords=SkyCoord(ra=gaia_data['ra'].to_numpy()*u.deg,dec=gaia_data['dec'].to_numpy()*u.deg)

In [59]:
montreal_df['ra_deg'] = coords.ra.deg
montreal_df['dec_deg']= coords.dec.deg

In [60]:
idx, sep2d,_ = coords.match_to_catalog_sky(gaia_coords)

In [61]:
max_separation = 1.0*u.arcsec
matches_within_radius = sep2d < max_separation

In [62]:
matched_gaia = gaia_data.iloc[idx[matches_within_radius]].reset_index(drop=True)
matched_montreal = montreal_df[matches_within_radius].reset_index(drop=True)